# Football Match Outcome Prediction - Project
# Milestone 2: Feature Engineering

In [3]:
import pandas as pd
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## 2.1 Feature Engineering

Our aim this stage is to create new features for the football dataset for a model to learn from. Having more options to choose from during model training can lead to finding features that can have high impact on the models accuracy.

First let's get out cleaned dataset from this previous Milestone:

In [11]:
full_results_table = pd.read_csv("cleaned_football_datasets/cleaned_dataset_no_merge.csv")

full_results_table = full_results_table.drop(["Unnamed: 0"], axis='columns')
full_results_table

,Home_Team,Away_Team,Result,Link,Season,Round,League
0,Cannes,Montpellier,2-1,https://www.besoccer.com/match/as-cannes/montp...,1991,1,ligue_1
1,PSG,Nancy,2-1,https://www.besoccer.com/match/paris-saint-ger...,1991,1,ligue_1
2,Olympique Marseille,Nice,1-0,https://www.besoccer.com/match/olympique-marse...,1991,1,ligue_1
3,Metz,Lille,2-2,https://www.besoccer.com/match/metz/lillestrom...,1991,1,ligue_1
4,Sochaux,Stade Brestois,1-1,https://www.besoccer.com/match/sochaux/stade-b...,1991,1,ligue_1
...,...,...,...,...,...,...,...
131688,Erzgebirge Aue,FC St. Pauli,1-3,https://www.besoccer.com/match/fc-erzgebirge-a...,2021,28,2_liga
131689,Paderborn,VfL Bochum,3-0,https://www.besoccer.com/match/paderborn/bochu...,2021,28,2_liga
131690,Hannover 96,Heidenheim,1-3,https://www.besoccer.com/match/hannover-96/hei...,2021,28,2_liga
131691,VfL Osnabrück,Braunschweig,0-4,https://www.besoccer.com/match/vfl-osnabruck/e...,2021,28,2_liga


In [48]:
# Extract ELO of each team
elo_list_raw = pickle.load(open('datasets/elo_dict.pkl', 'rb'))

In [49]:
# Convert elo_list_raw dictionary into pandas dataframe (transposed)
elo_list = pd.DataFrame.from_dict(pickle.load(open('datasets/elo_dict.pkl', 'rb'))).T.reset_index()

In [8]:
# Add features to dataframe

def add_elo(results_table, elo_list):
    '''
    Adds Elo_home and Elo_away columns to dataframe for each match for given results_table.

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Season,
        Round and League.

    Returns
    -------
    results_table_elo: pandas.core.frame.DataFrame
        Merges input results_table dataframe with global dataframe elo_list, which contains 
        values for Elo_home and Elo_away for each match. Uses equivalent values in Link
        and index columns to merge dataframes.
    '''
    results_table_elo = pd.merge(results_table, elo_list, left_on="Link", right_on="index").drop("index", axis=1) # .drop index column as duplicate values with link column
    return results_table_elo

def home_away_goals(results_table):
    '''
    Adds Home_Goals and Away_Goals columns to dataframe. Uses the Results column and inputs 
    values for number of home goals and away goals for each match for given results_table. 

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Input results_table with additional columns stating the number of Home and Away goals.
    '''

    home_result = []
    away_result = []

    # iterate through each value in Result column and splits into home and away goals
    for results in results_table["Result"]:
        home_result.append(int(results[:results.find('-')]))
        away_result.append(int(results[results.find('-')+1:]))

    # create new columns Home_Goals and Away_Goals
    results_table["Home_Goals"] = home_result
    results_table["Away_Goals"] = away_result
    
    return results_table

def win_loss_draw(results_table):
    '''
    Adds Label column to results_table which dictates if match result was Home Win, Away Win
    or a Draw.

    *IMPORTANT* Requires Home_Goals and Away_Goals columns in input results_table which can be 
    generated using the home_away_goals function. (This function will automatically run the home_away_goals 
    function on input results_table if Home_Goals and Away_Goals columns not found).
    Args

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Input results_table with additional columns stating the number of Home and Away goals.
    '''

    # if Home_Goals and Away_Goals columns are not in the results_table, generate them here
    if "Home_Goals" and "Away_Goals" not in results_table:
        home_away_goals(results_table)

    #define conditions for win or loss
    conditions = [results_table["Home_Goals"] > results_table["Away_Goals"], 
                results_table["Home_Goals"] < results_table["Away_Goals"]]

    # define choices : 1-Home Win, -1-Away Win, 0-Draw
    choices = [1, -1]

    #create new column in DataFrame that displays results of comparisons
    results_table["Label"] = np.select(conditions, choices, default=0)

    return results_table

def number_of_teams(results_table):
    '''
    Adds Number_Teams feature to results_table. This feature outputs the number of teams
    playing for the given league by counting the number of unique values in the Home_Team column.

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Append to input results_table with additional column Number_Teams; number of unique teams in
        results_table.
    '''
    results_table["Number_teams"] = len(results_table["Home_Team"].unique())
    return results_table

def total_rounds(results_table):
    '''
    Adds Total_Rounds feature to results_table. This feature outputs the total number of match 
    rounds in given league using the Round column.

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Append to input results_table with additional column Total_rounds; max number of rounds in
        results_table.
    '''
    results_table["Total_rounds"] = max(results_table["Round"])
    return results_table

def points_position(results_table):
    '''
    Adds four new features to results_table relating to points and position including:
        - Points_Home: Current number of match points for the Home Team 
        - Points_Away: Current number of match points for the Away Team 
        - Position_Home: Current league position of the Home Team 
        - Position_Away: Current league position of the Away Team 

    *IMPORTANT* Requires Home_Goals and Away_Goals columns in input results_table which can be 
    generated using the home_away_goals function. (This function will automatically run the home_away_goals 
    function on input results_table if Home_Goals and Away_Goals columns not found).

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Append to input results_table with additional columns Points_Home, Points_Away, 
        Position_Home, Position_Away.
    '''
    # if Home_Goals and Away_Goals columns are not in the results_table, generate them here
    if "Home_Goals" and "Away_Goals" not in results_table:
        home_away_goals(results_table)

    # map each team in given league with number of points and position. This will track their points and position throughout each game played
    team_points = {team : 0 for team in results_table["Home_Team"].unique()}
    position = {team : 0 for team in results_table["Home_Team"].unique()}

    # initialise dataframe which will state the number of points the Home and Away team has before going into a game, and respective position
    points_final = pd.DataFrame(
        {"Points_Home" : [],
        "Points_Away" : [],
        "Position_Home" : [],
        "Position_Away" : []
        }
    )

    # loop through each record in the results_table
    for index, row in results_table.iterrows():
        
        # order teams from highest to lowest points -- this will help determine their current position 
        points_ordered = {k: v for k, v in sorted(team_points.items(), key=lambda item: item[1], reverse=True)}
 
        # new dataframe which contains number of points and positions for the Home and Away team for given game 
        current_points = pd.DataFrame(
                {"Points_Home" : [team_points[row['Home_Team']]],
                "Points_Away" : [team_points[row['Away_Team']]],
                "Position_Home" : [position[row['Home_Team']]],
                "Position_Away" : [position[row['Away_Team']]]
                }
            )

        # Win +3, Loss +0, Draw +1
        if row['Home_Goals'] > row['Away_Goals']:
            team_points[row['Home_Team']] += 3

        elif row['Home_Goals'] == row['Away_Goals']:
            team_points[row['Home_Team']] += 1
            team_points[row['Away_Team']] += 1

        else:
            team_points[row['Away_Team']] += 3

        # find position of team based on number of points
        ### TO DO : ADD IN CONDITIONAL -- IF POINTS ARE THE SAME DETERMINE POSITION BASED ON GOAL DIFFERENCE ###
        if row["Round"] > 1:
            for index, team in enumerate(points_ordered):
                position[team] = index+1


        # Append new points values to end of dataframe
        # reset_index(drop=True) helps avoid InvalidIndexError
        points_final = pd.concat([points_final, current_points]).reset_index(drop=True)

    # Append Points_Home, Points_Away, Position_Home and Position_Away columns to results_table for each record
    results_table = pd.concat([results_table, points_final], axis=1)

    return results_table

def total_goals(results_table):
    '''
    Adds four new features to results_table relating to goals including:
        - Total_Goals_For_Home_Team: Total number of goals scored by Home Team (up to given game)
        - Total_Goals_Against_Home_Team: Total number of goals scored against Home Team (up to given game)
        - Total_Goals_For_Away_Team: Total number of goals scored by Away Team (up to given game)
        - Total_Goals_Against_Away_Team: Total number of goals scored against Away Team (up to given game)

    *IMPORTANT* Requires Home_Goals and Away_Goals columns in input results_table which can be 
    generated using the home_away_goals function. (This function will automatically run the home_away_goals 
    function on input results_table if Home_Goals and Away_Goals columns not found).

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Append to input results_table with additional columns Total_Goals_For_Home_Team, Total_Goals_Against_Home_Team, 
        Total_Goals_For_Away_Team, Total_Goals_Against_Away_Team.
    '''

    # if Home_Goals and Away_Goals columns are not in the results_table, generate them here
    if "Home_Goals" and "Away_Goals" not in results_table:
        home_away_goals(results_table)

    # dictionary which stores the total goals scored by each team and total goals scored against each team 
    goals_for = {team : 0 for team in results_table["Home_Team"].unique()}
    goals_against = {team : 0 for team in results_table["Home_Team"].unique()}

    # initialise dataframe which will state the number of goals the Home and Away team have scored/conceded before going into a game
    goals_final = pd.DataFrame(
        {"Total_Goals_For_Home_Team" : [],
        "Total_Goals_Against_Home_Team"  : [],
        "Total_Goals_For_Away_Team" : [],
        "Total_Goals_Against_Away_Team" : []
        }
    )

     # loop through each record in the results_table
    for index, row in results_table.iterrows():

        # new dataframe which contains number of goals for the Home and Away team for particular game 
        current_goals = pd.DataFrame(
                {"Total_Goals_For_Home_Team" : [goals_for[row['Home_Team']]],
                "Total_Goals_Against_Home_Team"  : [goals_against[row['Home_Team']]],
                "Total_Goals_For_Away_Team" : [goals_for[row['Away_Team']]],
                "Total_Goals_Against_Away_Team" : [goals_against[row['Away_Team']]]
                }
            )

        # add number of goals scored in game to the total number of goals scored for each team
        goals_for[row['Home_Team']] += row['Home_Goals']
        goals_for[row['Away_Team']] += row['Away_Goals']

        # add number of goals conceded in game to the total number of goals conceded for each team
        goals_against[row['Home_Team']] += row['Away_Goals']
        goals_against[row['Away_Team']] += row['Home_Goals']


        # Append new goals values to end of dataframe
        # reset_index(drop=True) helps avoid InvalidIndexError
        goals_final = pd.concat([goals_final, current_goals]).reset_index(drop=True)

    # Append Total_Goals_For_Home_Team, Total_Goals_For_Away_Team, Total_Goals_Against_Home_Team and Total_Goals_Against_Away_Team to results_table for each record
    results_table = pd.concat([results_table, goals_final], axis=1)

    return results_table

def total_result(results_table):
    '''
    Adds eight new features to results_table relating to a teams numbers of wins, draws and losses including:
        - Total_Wins_Home_Team: Total number of wins for Home Team
        - Total_Draws_Home_Team: Total number of draws for Home Team
        - Total_Loss_Home_Team: Total number of losses for Home Team
        - Total_Wins_Away_Team: Total number of wins for Away Team
        - Total_Draws_Away_Team: Total number of draws for Away Team
        - Total_Loss_Away_Team: Total number of losses for Away Team
        - Wins_When_Home: Total number of Home wins for Home Team
        - Draws_When_Home: Total number of Home draws for Home Team
        - Losses_When_Home: Total number of Home losses for Home Team
        - Wins_When_Away: Total number of Away wins for Away Team
        - Draws_When_Away: Total number of Away draws for Away Team
        - Losses_When_Away: Total number of Away losses for Away Team

    *IMPORTANT* Requires Label column in input results_table which can be generated using the win_loss_draw function. 
    (This function will automatically run the win_loss_draw function on input results_table if Label column not found).

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Label, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Append to input results_table with additional columns "Total_Wins_Home_Team, Total_Draws_Home_Team, 
        Total_Loss_Home_Team, Total_Wins_Away_Team, Total_Draws_Away_Team, Total_Loss_Away_Team, Wins_When_Home,
        Draws_When_Home, Losses_When_Home, Wins_When_Away, Draws_When_Away, Losses_When_Away.
    '''

    # if Label column is not in the results_table, generate it here
    if "Label" not in results_table:
        win_loss_draw(results_table)

    # multiple dictionaries which store the total number of wins, draws and losses for each team when Home and Away
    wins_home = {team : 0 for team in results_table["Home_Team"].unique()}
    draws_home = {team : 0 for team in results_table["Home_Team"].unique()}
    losses_home = {team : 0 for team in results_table["Home_Team"].unique()}

    wins_away = {team : 0 for team in results_table["Home_Team"].unique()}
    draws_away = {team : 0 for team in results_table["Home_Team"].unique()}
    losses_away = {team : 0 for team in results_table["Home_Team"].unique()}


    # initialise dataframe which will state total number of wins, draws and losses prior to current game for Home and Away team
    # Also states number of distinct wins, draws and losses for the Home Team when playing at Home and Away Team when playing Away
    result_final = pd.DataFrame(
        {"Total_Wins_Home_Team" : [],
        "Total_Draws_Home_Team"  : [],
        "Total_Loss_Home_Team" : [],
        "Total_Wins_Away_Team" : [],
        "Total_Draws_Away_Team"  : [],
        "Total_Loss_Away_Team" : [],
        "Wins_When_Home" :[],
        "Draws_When_Home" : [],
        "Losses_When_Home" : [],
        "Wins_When_Away" :[],
        "Draws_When_Away" : [],
        "Losses_When_Away" : []
        }
    )

     # loop through each record in the results_table
    for index, row in results_table.iterrows():

        # new dataframe which appends current total number of wins, draws and losses 
        # Total_Wins for Home and Away team (irrespective of if they are playing Home and Away) is calculated by adding their respective wins_home 
        # and wins_away values. (Similar for Draws and Losses)
        current_result = pd.DataFrame(
                {"Total_Wins_Home_Team" : [wins_home[row['Home_Team']] + wins_away[row['Home_Team']]],
                "Total_Draws_Home_Team"  : [draws_home[row['Home_Team']] + draws_away[row['Home_Team']]],
                "Total_Loss_Home_Team" : [losses_home[row['Home_Team']] + losses_away[row['Home_Team']]],
                "Total_Wins_Away_Team" : [wins_home[row['Away_Team']] + wins_away[row['Away_Team']]],
                "Total_Draws_Away_Team" : [draws_home[row['Away_Team']] + draws_away[row['Away_Team']]],
                "Total_Loss_Away_Team" : [losses_home[row['Away_Team']] + losses_away[row['Away_Team']]],
                "Wins_When_Home" : [wins_home[row['Home_Team']]],
                "Draws_When_Home" : [draws_home[row['Home_Team']]],
                "Losses_When_Home" : [losses_home[row['Home_Team']]],
                "Wins_When_Away" :[wins_away[row['Away_Team']]],
                "Draws_When_Away" : [draws_away[row['Away_Team']]],
                "Losses_When_Away" : [losses_away[row['Away_Team']]]
                }
            )

        # if the Home Team wins, add 1 to wins_home dictionary for the Home Team key, add 1 to losses_away dictionary for Away Team key
        if row['Label'] == 1:
            wins_home[row['Home_Team']] += 1
            losses_away[row['Away_Team']] += 1
        # if match result is draw, add 1 for both teams in draws_home and draws_away dictionary correspondingly
        elif row['Label'] == 0:
            draws_home[row['Home_Team']] += 1
            draws_away[row['Away_Team']] += 1
        # if the Away Team wins, add 1 to wins_away dictionary for Away Team key, add 1 to losses_home dictionary for Home Team key
        elif row['Label'] == -1:
            losses_home[row['Home_Team']] += 1
            wins_away[row['Away_Team']] += 1
            



        # Append new results values to end of dataframe
        # reset_index(drop=True) helps avoid InvalidIndexError
        result_final = pd.concat([result_final, current_result]).reset_index(drop=True)

    # Append all new created features to given results_table
    results_table = pd.concat([results_table, result_final], axis=1)

    return results_table

def total_streaks(results_table):
    '''
    Adds four new features to results_table relating to a teams current winning streak including:
        - Total_Streak_Home_Team: Total winning streak for Home Team including both Home and Away games.
                                  Any draw or loss, will result in streak reset.
        - Total_Streak_Away_Team: Total winning streak for Home Team including both Home and Away games.
                                  Any draw or loss, will result in streak reset.
        - Streak_When_Home: Total winning streak for Home Team when playing Home i.e. not including any Away games.
                                  Any draw or loss while team playing Home, will result in streak reset.
        - Streak_When_Away: Total winning streak for Away Team when playing Away i.e. not including any Home games.
                                  Any draw or loss while team playing Away, will result in streak reset.

    *IMPORTANT* Requires Label column in input results_table which can be generated using the win_loss_draw function. 
    (This function will automatically run the win_loss_draw function on input results_table if Label column not found).

    Args
    ----------
    results_table: pandas.core.frame.DataFrame
        Dataframe of match results containing the Home_Team, Away_Team, Result, Link, Label, Season,
        Round, League (and more).

    Returns
    -------
    results_table: pandas.core.frame.DataFrame
        Append to input results_table with additional columns Total_Streak_Home_Team, Total_Streak_Away_Team,
        Streak_When_Home, Streak_When_Away.
    '''

    # if Label column is not in the results_table, generate it here
    if "Label" not in results_table:
        win_loss_draw(results_table)


    # multiple dictionaries which store winning streaks of each team. The total streak, and the streak when playing Home or Away correspondingly
    streak_total = {team : 0 for team in results_table["Home_Team"].unique()}

    streak_when_home = {team : 0 for team in results_table["Home_Team"].unique()}

    streak_when_away = {team : 0 for team in results_table["Home_Team"].unique()}


    # initialise dataframe which will state winning streaks of each team 
    streak_final = pd.DataFrame(
        {"Total_Streak_Home_Team" : [],
        "Total_Streak_Away_Team" : [],
        "Streak_When_Home" : [],
        "Streak_When_Away" : []
        }
    )

     # loop through each record in the results_table
    for index, row in results_table.iterrows():

        # new dataframe which appends current streaks
        current_streak = pd.DataFrame(
                {"Total_Streak_Home_Team" : [streak_total[row["Home_Team"]]],
                "Total_Streak_Away_Team" : [streak_total[row["Away_Team"]]],
                "Streak_When_Home" : [streak_when_home[row["Home_Team"]]],
                "Streak_When_Away" : [streak_when_away[row["Away_Team"]]]
                }
            )

        # if Home Team wins, reset Away Team streaks for total and away to 0 and +1 to total streak and streak when home for home team
        if row['Label'] == 1:
            streak_total[row["Home_Team"]] += 1
            streak_total[row["Away_Team"]] = 0
            
            streak_when_home[row["Home_Team"]] += 1
            streak_when_away[row["Away_Team"]] = 0
        # if match result is draw, reset all streaks to 0
        elif row['Label'] == 0:
            streak_total[row['Home_Team']] = 0
            streak_total[row['Away_Team']] = 0

            streak_when_home[row["Home_Team"]] = 0
            streak_when_away[row["Away_Team"]] = 0


        # if Away Team wins, reset Home Team streaks for total and home to 0 and +1 to total streak and streak when away for away team
        elif row['Label'] == -1:
            streak_total[row['Home_Team']] = 0
            streak_total[row['Away_Team']] += 1

            streak_when_home[row["Home_Team"]] = 0
            streak_when_away[row["Away_Team"]] += 1
            

        # Append new results values to end of dataframe
        # reset_index(drop=True) helps avoid InvalidIndexError
        streak_final = pd.concat([streak_final, current_streak]).reset_index(drop=True)

    # Append all new created features to given results_table
    results_table = pd.concat([results_table, streak_final], axis=1)

    return results_table


In [16]:
import pandas as pd
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# additional datasets from external sources
elo_table = pd.DataFrame.from_dict(pickle.load(open('datasets/elo_dict.pkl', 'rb'))).T.reset_index()

# Pipeline to add in additional features created locally
full_results_table = number_of_teams(full_results_table)
full_results_table = total_rounds(full_results_table)
full_results_table = home_away_goals(full_results_table)
full_results_table = win_loss_draw(full_results_table)
full_results_table = add_elo(full_results_table, elo_table)
# full_results_table = total_streaks(full_results_table)
# full_results_table = total_result(full_results_table)
# full_results_table = points_position(full_results_table)
# full_results_table = total_goals(full_results_table)


full_results_table







,Home_Team,Away_Team,Result,Link,Season,Round,League,Number_teams,Total_rounds,Home_Goals,Away_Goals,Label,Elo_home,Elo_away
0,Cannes,Montpellier,2-1,https://www.besoccer.com/match/as-cannes/montp...,1991,1,ligue_1,503,46,2,1,1,74.0,74.0
1,PSG,Nancy,2-1,https://www.besoccer.com/match/paris-saint-ger...,1991,1,ligue_1,503,46,2,1,1,78.0,73.0
2,Olympique Marseille,Nice,1-0,https://www.besoccer.com/match/olympique-marse...,1991,1,ligue_1,503,46,1,0,1,85.0,73.0
3,Metz,Lille,2-2,https://www.besoccer.com/match/metz/lillestrom...,1991,1,ligue_1,503,46,2,2,0,76.0,75.0
4,Sochaux,Stade Brestois,1-1,https://www.besoccer.com/match/sochaux/stade-b...,1991,1,ligue_1,503,46,1,1,0,78.0,74.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131687,Erzgebirge Aue,FC St. Pauli,1-3,https://www.besoccer.com/match/fc-erzgebirge-a...,2021,28,2_liga,503,46,1,3,-1,55.0,57.0
131688,Paderborn,VfL Bochum,3-0,https://www.besoccer.com/match/paderborn/bochu...,2021,28,2_liga,503,46,3,0,1,57.0,61.0
131689,Hannover 96,Heidenheim,1-3,https://www.besoccer.com/match/hannover-96/hei...,2021,28,2_liga,503,46,1,3,-1,56.0,63.0
131690,VfL Osnabrück,Braunschweig,0-4,https://www.besoccer.com/match/vfl-osnabruck/e...,2021,28,2_liga,503,46,0,4,-1,52.0,52.0
